# 11장 자연어 처리

##  IMDB 영화 후기 데이터셋 벡터화

IMDB 데이터셋을 직접 다운로드하여 벡터화하는 과정을 살펴본다.

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다.

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5724k      0  0:00:14  0:00:14 --:--:-- 13.2M


`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [2]:
import platform

if platform.system() == 'Linux':
    !rm -r aclImdb/train/unsup
else:
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [3]:
if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)

    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기

    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]

    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여
훈련셋, 검증셋, 테스트셋을 준비한다.
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [5]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [6]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)

    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'In the days before gore and sex took over, real horror films were made. Castle of Blood is, in my estimation, one of the finest, although other reviewers have given it mixed ratings. In an odd sort of way it reminds of the more recent The Others, which was in the theaters a couple of years ago.<br /><br />Director Antonio Margheriti remade his own picture in 1970 titling it this time Web of the Spider (AKA Nella Stretta Morsa del Ragno). Why he did this I do not understand, although the remake starred Anthony Franciosa and Klaus Kinski and was very good in its own right. Perhaps he saw a good story and wished to tailor it more to American audiences. I do not really know. It is interesting that he did the original in black and white and the remake in color.<br /><br />Castle of Blood is excellent Italian Gothic. La Danza Macabra is said to be an unpublished wor

### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다.
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다.

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [7]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
# 어휘 색인 생성 대상 훈련셋 후기 텍스트 데이터셋
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다.
또한 벡터에 사용된 정수는 2만보다 작은 값이며,
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [8]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[  109  2304    11    20   348  1008    21  6988     3   248    11    64
  3618 15266     3     4   384   511    60    66     6  2647    17     2
  3224 10521     5     2   700  5367    41    39   105    83  1783  4574
    16   459 15320     2    85   189   332    14   127     3    81     8
    11   511    67   124  2142    37    65   408  1285     3     2  5362
    60   332   205  1986  3766 11558     1     1  1472  1664    37     2
   332    17     4   408  3676     3    14     2  1096   417  1763     8
   511    28    14   777    58     4  1101     3    14    22    53   656
    17     2  7258    81     3    35     8   456   115    22  1271  1742
    89   347    15     2  2156  1100  1945     6   685     1 19430  4934
     6   399     4  3224     8   644     6   923     3 12280    11   384
  7258   511    16    65  2200   896    89    60    14    63    32 

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

In [10]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 길이가 256인 1차원 어레이로 변환
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.3385 - accuracy: 0.8547INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 146s 233ms/step - loss: 0.3385 - accuracy: 0.8547 - val_loss: 0.3318 - val_accuracy: 0.8594
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.2989 - accuracy: 0.8723INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 141s 225ms/step - loss: 0.2989 - accuracy: 0.8723 - val_loss: 0.3204 - val_accuracy: 0.8634
Epoch 3/20
625/625 [==============================] - ETA: 0s - loss: 0.2680 - accuracy: 0.8893INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 142s 227ms/step - loss: 0.2680 - accuracy: 0.8893 - val_loss: 0.3118 - val_accuracy: 0.8706
Epoch 4/20
625/625 [==============================] - ETA: 0s - loss: 0.2386 - accuracy: 0.9039INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 142s 226ms/step - loss: 0.2386 - accuracy: 0.9039 - val_loss: 0.3090 - val_accuracy: 0.8688
Epoch 5/20
625/625 [==============================] - 141s 225ms/step - loss: 0.2071 - accuracy: 0.9178 - val_loss: 0.3177 - val_accuracy: 0.8736
Epoch 6/20
625/625 [==============================] - 138s 220ms/step - loss: 0.1833 - accuracy: 0.9293 - val_loss: 0.3281 - val_accuracy: 0.8756
Epoch 7/20
625/625 [==============================] - 138s 220ms/step - loss: 0.1550 - accuracy: 0.9396 - val_loss: 0.3379 - val_accuracy: 0.8718
Epoch 8/20
625/625 [==============================] - 138s 220ms/step - loss: 0.1288 - accuracy: 0.9514 - val_loss: 0.3661 - val_accuracy: 0.8638
Epoch 9/20
625/625 [==============================] - 136s 217ms/step - loss: 0.1069 - accuracy: 0.9627 - val_loss: 0.4106 - val_accuracy: 0.8592
Epoch 10/20
625/625 [==============================] - 138s 221ms/step - loss: 0.0873 - accuracy: 0.9678 - val_loss: 0.4019 - val_accur

**단어 위치 인코딩**

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다.
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가
단어위치를 활용할 수 있도록 한다.

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "full_transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.5303 - accuracy: 0.7409INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 212s 334ms/step - loss: 0.5303 - accuracy: 0.7409 - val_loss: 0.3390 - val_accuracy: 0.8530
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.2971 - accuracy: 0.8765INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 202s 322ms/step - loss: 0.2971 - accuracy: 0.8765 - val_loss: 0.3021 - val_accuracy: 0.8730
Epoch 3/20
625/625 [==============================] - ETA: 0s - loss: 0.2351 - accuracy: 0.9072INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 189s 302ms/step - loss: 0.2351 - accuracy: 0.9072 - val_loss: 0.2917 - val_accuracy: 0.8864
Epoch 4/20
625/625 [==============================] - 197s 315ms/step - loss: 0.1961 - accuracy: 0.9236 - val_loss: 0.4036 - val_accuracy: 0.8482
Epoch 5/20
625/625 [==============================] - 202s 323ms/step - loss: 0.1619 - accuracy: 0.9391 - val_loss: 0.4399 - val_accuracy: 0.8160
Epoch 6/20
625/625 [==============================] - 203s 324ms/step - loss: 0.1373 - accuracy: 0.9484 - val_loss: 0.5411 - val_accuracy: 0.8412
Epoch 7/20
625/625 [==============================] - 198s 317ms/step - loss: 0.1160 - accuracy: 0.9570 - val_loss: 0.3829 - val_accuracy: 0.8828
Epoch 8/20
625/625 [==============================] - 204s 327ms/step - loss: 0.0996 - accuracy: 0.9639 - val_loss: 0.3933 - val_accuracy: 0.8860
Epoch 9/20
625/625 [==============================] - 194s 309ms/step - loss: 0.0800 - accuracy: 0.9712 - val_loss: 0.4089 - val_accura

## 시퀀스-투-시퀀스 학습

영어-스페인어 기계 번역과 영어-한국어 기계 번역을 작은 데이터셋을 이용하여 훈련시켜본다.

### 영어-스페인어 기계 번역

**텍스트 데이터셋 다운로드**

영어와 스페인어 텍스트가 담긴 압축 파일을 다운로드 한 후에 압축을 풀면
"spa.txt" 파일이 생성된다.

In [ ]:
!wget https://www.manythings.org/anki/spa-eng.zip
!unzip -q spa-eng.zip

--2023-12-16 18:29:23--  https://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5413153 (5.2M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   5.16M  3.99MB/s    in 1.3s    

2023-12-16 18:29:25 (3.99 MB/s) - ‘spa-eng.zip’ saved [5413153/5413153]



"spa.txt" 파일은 각각의 줄은 아래와 같이 영어 텍스트, 스페인어 텍스트, 기타 정보가 탭(tab) 키로 구분되어 있다.

```
Finally, it's Friday.	Al fin es viernes.	CC-BY 2.0 (France) Attribution: tatoeba.org #433868 (CK) & #1427385 (marcelostockle)
```

아래 코드는 "spa.txt"에 포함된 각 줄의 내용을 항목으로 갖는 리스트인 `text_pairs`를 생성한다.
단 각 항목은 (영어 텍스트, 스페인어 텍스트)로 구성된 튜플이며, 각각의 줄에 포함된 기타 정보는 버린다.
또한 스페인어 텍스트의 처음과 끝에 각각 `'[start] '` 와 `' [end]'`를 추가한다.

In [ ]:
text_file = "spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, spanish, _ = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

`text_pairs`에 포함된 임의의 항목을 확인하면 다음과 같다.

In [ ]:
import random

print(random.choice(text_pairs))

('You must be more polite.', '[start] Tienes que ser más educado. [end]')


아래 코드는 텍스트를 무작위 섞은 다음
70 대 15 대 15의 비율로 훈련 텍스트셋, 검증 텍스트셋, 테스트 텍스트셋으로 나눈다.

In [ ]:
random.shuffle(text_pairs)

# 검증셋 크기: 전체 데이터셋의 15%
num_val_samples = int(0.15 * len(text_pairs))
# 훈련셋 크기: 전체 데이터셋의 70%
num_train_samples = len(text_pairs) - 2 * num_val_samples

# 훈련 텍스트셋
train_pairs = text_pairs[:num_train_samples]
# 검증 텍스트셋
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
# 테스트 텍스트셋
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**영어/스페인어 텍스트 벡터화**

자연어로 구성된 훈련 텍스트 데이터셋을 대상으로 어휘 인덱스를 생성한 후에 텍스트 벡터화를 진행한다.
먼저 영어 어휘집을 생성한다.
생성되는 어휘 벡터의 길이를 20으로 지정한다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

vocab_size = 15000
sequence_length = 20

# 번역 대상 언어(예를 들어 영어) 텍스트 데이터셋 벡터화 층
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# 영어 텍스트만 추출
train_english_texts = [pair[0] for pair in train_pairs]
# 영어 어휘집 생성
source_vectorization.adapt(train_english_texts)

스페인어 텍스트 벡터화는 영어와는 다른 표준화 방식을 사용한다.

- 영어에는 없는 `'¿'` 기호도 표준화 과정에서 삭제
- 반면에 `'['`와 `']'`는 표준화 과정에서 제거되지 않도록 지정

또한 생성되는 어휘 벡터의 길이를 21로 지정한다.
그러면 0번 인덱스부터 19번 인덱스까지는 입력값으로,
1번 인덱스부터 20번 인덱스까지는 타깃으로 지정할 수 있다.

In [ ]:
import string
import re

# 마침표 기호 목록에 "¿" 추가. 즉 표준화과정에서 삭제 대상으로 지정.
strip_chars = string.punctuation + "¿"
# 마침표 기호 목록으로부터 "[" 와 "]" 제거. 즉, 표준화 대상에서 삭제하지 않도록 함.
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

# 새로운 표준화 함수 선언
# 소문자로 변환한 후에 strip_chars 에 포함된 모든 기호 삭제
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# 번역 언어 (예를 들어 스페인어) 텍스트 데이터셋 벡터화 층
# 벡터의 길이를 20이 아닌 21로 지정. 입력값과 타깃을 구분하기 위해 필요함.
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

# 스페인어 텍스트만 추출
train_spanish_texts = [pair[1] for pair in train_pairs]
# 스페인어 어휘집 생성
target_vectorization.adapt(train_spanish_texts)

훈련 텍스트셋, 검증 텍스트셋, 테스트 텍스트셋을 모두 어휘 인덱스를 이용하여 벡터화 한다.

아래 코드는 생성된 영어와 스페인어 어휘 인덱스를 이용하여 각각의 텍스트 데이터셋을
벡터화 한 다음에 아래 모양의 튜플로 구성된 훈련셋, 검증셋, 테스트셋을 생성한다.

- 튜플의 첫째 항목: 영어 입력 배치와 스페인어 입력 배치로 구성된 사전. 모델의 입력값으로 사용.
- 튜플의 둘째 항목: 타깃 배치. 모델 훈련의 타깃으로 사용.

```
({"english": 영어 입력 배치, "spanish": 스페인어 입력 배치}, 타깃 배치)
```

- `format_dataset()` 함수
    - 인자: 영어 텍스트 배치와 스페인어 텍스트 배치
    - 반환값: 앞서 언급한 모양의 사전
    
- `make_dataset()` 함수
    - 인자: `(영어 텍스트, 스페인어 텍스트)` 모양의 튜플로 구성된 자연어 텍스트 데이터셋\
    - 반환값: 지정된 배치 크기로 묶은 배치들에 대해 `format_dataset()` 함수를 적용하여
        생성된 `Dataset` 자료형의 데이터셋. 배치 단위로 묶여 있음.
        - `dataset.shuffle(2048).prefetch(16).cache()`: 대용량 데이터셋을 배치 단위로
            빠르게 불러오기 위해 사용함.        

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({"english": eng, "spanish": spa[:, :-1]}, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)

    dataset = dataset.map(format_dataset)

    return dataset.shuffle(2048).prefetch(16).cache()

# 훈련셋
train_ds = make_dataset(train_pairs)
# 검증셋
val_ds = make_dataset(val_pairs)
# 테스트셋
test_ds = make_dataset(test_pairs)

예를 들어 훈련셋의 첫째 배치의 모양을 확인하면 다음과 같다.

- 영어 입력 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 영어 텍스트 64개로 구성됨.
- 스페인어 입력 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 스페인어 텍스트 64개로 구성됨.
- 타깃 배치: 길이가 20인 64개의 벡터로 구성. 즉, 20 개의 단어로 구성된 스페인어 텍스트 64개로 구성됨.

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


아래 코드는 첫째 샘플의 영어 벡터, 스페인어 벡터, 타깃을 보여준다.
스페인어 입력 벡터 샘플의 0번 인덱스에 위치한 정수 2가 `'[start]'`에 해당하는 값이다.
스페인어 타깃 벡터 샘플은 그 값을 제외한 벡터로 시작함을 확인할 수 있다.
또한 정수 3은 `'[end]'`에 해당하는 값이며, 문장의 끝을 가리키기에
스페인어 타깃 벡터 샘플에 새로운 단어에 해당하는 인덱스를 추가하지 않고 대신 0 패딩이 하나 더 추가되었다.

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"영어 입력 벡터 샘플: {inputs['english'][0]}")
    print(f"스페인어 입력 벡터 샘플: {inputs['spanish'][0]}")
    print(f"스페인어 타깃 샘플: {targets[0]}")

영어 입력 벡터 샘플: [  3 351   4 738   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
스페인어 입력 벡터 샘플: [   2 2208    4 5167    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
스페인어 타깃 샘플: [2208    4 5167    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


### 트랜스포머 디코더

트랜스포머 디코더를 하나의 층으로 구현하면 다음과 같다.
생성자의 인자는 다음과 같다.

- `embed_dim`: 예를 들어 `embed_dim=256`은 단어 임베딩 `(600, 256)` 모양의 샘플 생성
- `dense_dim`: 밀집층에서 사용되는 유닛<font size='2'>unit</font> 개수
- `num_heads`: 헤드<font size='2'>head</font> 개수

`get_causal_attention_mask()` 메서드는 스페인어 입력 텍스트에 대한 마스크를 지정할 때 활용되지만
여기서는 마스크를 사용하지 않는다.

순전파를 담당하는 `call()` 메서드는 두 개의 어텐션 층을 사용한다.
입력값으로는 스페인어 텍스트 배치 데이터셋과
트랜스포머 디코더의 출력값으로 셀프 어텐션이 적용되어 변환된 영어 텍스트 배치 데이터셋이 사용된다.

- `attention_1`: 스페인어 텍스트 입력값에 대해 셀프 어텐션 적용
- `attention_2`: `attention_1` 의 출력값을 query로, 트랜스포머 인코더의 출력값을 key와 value로 사용해서 어텐션 적용.

최종적으로 두 개의 밀집층을 통과시킨다.
또한 하나의 블록을 통과시킬 때마다 잔차연결과 층정규화를 진행한다.

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # 마스크 활용
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        # 셀프 어텐션 적용: 번역 언어(예를 들어 스페인어) 입력값 대상
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        # 셀프 어텐션이 적용된 (예를 들어 스페인어) 입력 텍스트를 query로
        # 셀프 어텐션이 적용된 번역 대상 (예를 들어 영어) 입력 텍스트를 key와 value로
        # 지정하여 어텐션 적용
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)

        proj_output = self.dense_proj(attention_output_2)

        return self.layernorm_3(attention_output_2 + proj_output)

### 기계 번역 모델

모델의 입력값은 앞서 설명한 대로 예를 들어 일정 길이로 단어 벡터화된 영어 텍스트 데이터셋과
스페인어 텍스트 데이터셋의 튜플이다.
스페인어 텍스트는 모두 `[start]` 로 시작하도록 전처리되어 있다.

모델의 출력값은 예를 들어 출력 스페인어 텍스트로 지정될 단어들에 대한 위치별 확률값을 계산한다.
아래 코드에서는 스페인어 텍스트에 포함될 20 개 단어들의 후보를 위치별로 확률값으로 계산한다.
예를 들어 출력 텍스트의 i-번 인덱스에 위치할 단어의 확률값을 계산하기 위해
어휘집에 포함된 15,000 개 단어를 대상으로 각각의 단어가 해당 위치에 자리할 확률을
소프트맥스 함수를 이용하여 계산한다.

In [ ]:
sequence_length = 20 # 텍스트의 단어수
vocab_size = 15000 # 어휘집 크기
embed_dim = 256    # 단어 임베딩 크기
dense_dim = 2048   # 밀집층 유닛수
num_heads = 8      # 어텐션 헤드수

# 트랜스포머 인코더 활용

# 첫째 입력값: 예를 들어 영어 텍스트셋
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 트랜스포머 디코더 활용

# 둘째 입력값: 예를 들어 스페인어 텍스트셋
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**모델 훈련과 활용**

모델의 최종 출력값이 소프트맥스를 사용하여
`(20, 15000)` 모양을 갖는 반면에
타깃셋은 20 개의 어휘 인덱스로 구성된 벡터로 구성되기에
`categorical_crossentropy` 가 아닌 `sparse_categorical_crossentropy`를
손실함수로 지정한다.
그러면 20개 단어 각각에 대해 가장 높은 확률을 갖는 (어휘) 인덱스에 해당하는 단어가
15,000 개 중에 선택되어 타깃 단어와 비교된다.

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1547/1547 [==============================] - 10640s 67ms/step - loss: 3.6921 - accuracy: 0.4524 - val_loss: 2.7772 - val_accuracy: 0.5561
Epoch 2/30
1547/1547 [==============================] - 80s 51ms/step - loss: 2.7889 - accuracy: 0.5603 - val_loss: 2.4589 - val_accuracy: 0.6021
Epoch 3/30
1547/1547 [==============================] - 78s 51ms/step - loss: 2.5259 - accuracy: 0.6007 - val_loss: 2.3711 - val_accuracy: 0.6203
Epoch 4/30
1547/1547 [==============================] - 79s 51ms/step - loss: 2.3810 - accuracy: 0.6247 - val_loss: 2.3070 - val_accuracy: 0.6347
Epoch 5/30
1547/1547 [==============================] - 79s 51ms/step - loss: 2.2894 - accuracy: 0.6414 - val_loss: 2.3228 - val_accuracy: 0.6362
Epoch 6/30
1547/1547 [==============================] - 79s 51ms/step - loss: 2.2183 - accuracy: 0.6549 - val_loss: 2.2714 - val_accuracy: 0.6467
Epoch 7/30
1547/1547 [==============================] - 80s 51ms/step - loss: 2.1518 - accuracy: 0.6684 - val_loss: 2.235

아래 `decode_sequence()`는 함수는 영어 텍스트가 하나 입력되면
앞서 훈련된 트랜스포머 모델을 이용하여 지정된 길이인 20 개의 단어로
구성된 스페인어 텍스트를 생성한다.

함수 본문에 포함된 `for` 반복문은
**트랜스포머 모델 활용** 부분에서 설명한 방식 그대로
`[start]`로만 구성된 텍스트로 시작해서
계속해서 텍스트에 추가할 단어를 하나씩 선택해서 이어가는 과정을
`[end]` 키워드가 나올 때까지 반복한다.
단, 반복횟수는 20으로 제한한다.

In [ ]:
import numpy as np

# 어휘집 확인
spa_vocab = target_vectorization.get_vocabulary()
# (단어 인덱스, 단어)로 구성된 사전 지정
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
# 텍스트에 포함되는 단어수
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    # 기계 번역 시작
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        # 트랜스포머 모델 적용
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])

        # i-번째 단어로 사용될 어휘 인덱스 확인
        sampled_token_index = np.argmax(predictions[0, i, :])
        # i-번째 단어 확인
        sampled_token = spa_index_lookup[sampled_token_index]
        # 스페인어 입력 텍스트에 i-번째 단어로 추가
        decoded_sentence += " " + sampled_token
        # 기계 번역 종료 조건 확인
        if sampled_token == "[end]":
            break

    return decoded_sentence

아래 코드는 `decode_sequence()` 함수를 이용하여
무작위로 5개의 영어 텍스트를 선택하여 기계 번역한 결과이다.

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
You should not despise a man because he is poor.
[start] no debes hacer pobre [UNK] a un pobre [end]
-
I'm calling you.
[start] te estoy todavía [end]
-
The influence of TV on society is great.
[start] la montaña de la gente de la [UNK] es muy importante [end]
-
Did Tom listen to you?
[start] tom te escuchando [end]
-
My plan was eventually adopted.
[start] mi plan fue al final [end]
